# Get own implied volatilities and compare them to Deribits

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', 500)
import seaborn as sns
import plotly.graph_objects as go
import datetime
from scipy import stats as sps

from utils import get_human_timestamp, get_difference_between_now_and_expirety_date
import plotly
import plotly.graph_objs as go
from typing import Union
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv(f"./datasets/deribit_options_chain_2022-12-01_OPTIONS.csv")
data = df.copy()

In [27]:
# Newton-Raphsen
def get_implied_volatility_call(
    option_type: Union["call", "put"],
    C: float,
    K: float,
    T: float,
    F: float,
    r: float = 0.0,
    error: float = 0.001,
) -> float:
    """
    Function to count implied volatility via given params of option, using Newton-Raphson method :

    Args:
        C (float): Option market price(USD).
        K (float): Strike(USD).
        T (float): Time to expiration in years.
        F (float): Underlying price.
        r (float): Risk-free rate.
        error (float): Given threshhold of error.

    Returns:
        float: Implied volatility in percent.
    """
    vol = 1.0
    dv = error + 1
    while abs(dv) > error:
        d1 = (np.log(F / K) + 0.5 * vol**2 * T) / (vol * np.sqrt(T))
        d2 = d1 - vol * np.sqrt(T)
        D = np.exp(-r * T)
        if option_type.lower() == "call":
            price = F * sps.norm.cdf(d1) - K * sps.norm.cdf(d2) * D
        elif option_type.lower() == "put":
            price = -F * sps.norm.cdf(-d1) + K * sps.norm.cdf(-d2) * D
        else:
            raise ValueError("Wrong option type, must be 'call' or 'put' ")
        Vega = F * np.sqrt(T / np.pi / 2) * np.exp(-0.5 * d1**2)
        PriceError = price - C
        dv = PriceError / Vega
        vol = vol - dv
    # in percents, as deribit shows
    return vol * 100

In [167]:
test = data.sample(10)
test[["symbol", "type", "mark_price", "underlying_price", "strike_price", "expiration", "delta", "mark_iv"]]

,symbol,type,mark_price,underlying_price,strike_price,expiration,delta,mark_iv
7267981,ETH-31MAR23-1900-C,call,0.0477,1277.3600,1900,1680249600000000,0.23392,73.85
5756648,BTC-30JUN23-100000-P,put,4.8583,17081.9800,100000,1688112000000000,-0.98048,95.73
6186346,ETH-9DEC22-600-P,put,0.0001,1281.9900,600,1670572800000000,-0.00153,180.96
20249085,BTC-30DEC22-20000-P,put,0.1932,16891.7600,20000,1672387200000000,-0.86494,51.51
14019425,ETH-30JUN23-6500-P,put,4.1013,1276.0400,6500,1688112000000000,-0.96712,97.02
10605305,SOL-30DEC22-28-C,call,0.0125,13.3441,28,1672387200000000,0.07431,158.21
19861400,ETH-30DEC22-2500-P,put,0.9680,1271.2500,2500,1672387200000000,-0.98483,104.77
9131414,BTC-24FEB23-35000-P,put,1.0525,17083.3856,35000,1677225600000000,-0.96911,72.78
6040167,ETH-30DEC22-8000-C,call,0.0000,1283.8600,8000,1672387200000000,0.00017,169.73
4924816,ETH-24FEB23-1300-P,put,0.1564,1277.1489,1300,1677225600000000,-0.44598,76.22


# In USD

In [168]:
for index, row in test.iterrows():
    iv = get_implied_volatility_call(
        option_type = row["type"], 
        C = row["mark_price"]*row["underlying_price"],
        K = row["strike_price"],
        T = get_difference_between_now_and_expirety_date(row["expiration"], row["timestamp"]),
        F = row["underlying_price"],
        r = 0.0,
        error = 0.001
    )
    deribit_value = row["mark_iv"]
    print(f"""Deribit IV: {deribit_value}, counted IV: {iv}, error: {2*(deribit_value - iv)/(iv + deribit_value) * 100}% """)

Deribit IV: 73.85, counted IV: 73.84847236869476, error: 0.0020685810499426627% 
Deribit IV: 95.73, counted IV: 95.74169892253133, error: -0.01222000180408558% 
Deribit IV: 180.96, counted IV: nan, error: nan% 
Deribit IV: 51.51, counted IV: 51.449074334972835, error: 0.11834928668636678% 
Deribit IV: 97.02, counted IV: 96.84757595022681, error: 0.17787817166234335% 
Deribit IV: 158.21, counted IV: 158.12336535724245, error: 0.05477426806351636% 
Deribit IV: 104.77, counted IV: 104.70199576255546, error: 0.06492919227410815% 
Deribit IV: 72.78, counted IV: 72.71053735339548, error: 0.0954875112403997% 
Deribit IV: 169.73, counted IV: 34.71946387975734, error: 132.07228188149838% 
Deribit IV: 76.22, counted IV: 76.21061879803446, error: 0.012308815695314607% 


# In crypto

In [169]:
for index, row in test.iterrows():
    iv = get_implied_volatility_call(
        option_type = row["type"], 
        C = row["mark_price"],
        K = row["strike_price"]/row["underlying_price"],
        T = get_difference_between_now_and_expirety_date(row["expiration"], row["timestamp"]),
        F = row["underlying_price"]/row["underlying_price"], # always 1.0
        r = 0.0,
        error = 0.001
    )
    deribit_value = row["mark_iv"]
    print(f"""Deribit IV: {deribit_value}, counted IV: {iv}, error: {2*(deribit_value - iv)/(iv + deribit_value) * 100}% """)

Deribit IV: 73.85, counted IV: 73.84847236869476, error: 0.0020685810499426627% 
Deribit IV: 95.73, counted IV: 95.74169892253398, error: -0.012220001806861206% 
Deribit IV: 180.96, counted IV: nan, error: nan% 
Deribit IV: 51.51, counted IV: 51.449074334972444, error: 0.11834928668712635% 
Deribit IV: 97.02, counted IV: 96.8475759502265, error: 0.17787817166266615% 
Deribit IV: 158.21, counted IV: 158.12336535724245, error: 0.05477426806351636% 
Deribit IV: 104.77, counted IV: 104.70199576255621, error: 0.06492919227338878% 
Deribit IV: 72.78, counted IV: 72.7105373533962, error: 0.09548751123940295% 
Deribit IV: 169.73, counted IV: 34.719463879748574, error: 132.0722818815126% 
Deribit IV: 76.22, counted IV: 76.21061879803442, error: 0.012308815695370547% 


## Result: For out of the money options deribit counts IV's via BSM. For deep in the money they give strange values which are impossible to get via Newton-Raphsen method

## Interesting to take a single tick and find the minimum of the error, which will help to find out the risk-free rate, whick deribit takes

In [218]:
# for out of the money calls
# test = data.sample(1000)
tick = (
    test[(test["type"] == "call") & (test["underlying_price"] < test["strike_price"])]
    .sample(1)
    .iloc[0]
)
rs = np.linspace(0.0, 0.001, 1000)
errors = []
for r in rs:
    iv = get_implied_volatility_call(
        option_type=tick["type"],
        C=tick["mark_price"],
        K=tick["strike_price"] / tick["underlying_price"],
        T=get_difference_between_now_and_expirety_date(
            tick["expiration"], tick["timestamp"]
        ),
        F=tick["underlying_price"] / tick["underlying_price"],  # always 1.0
        r=r,
        error=0.001,
    )
    deribit_value = tick["mark_iv"]
    errors.append(abs(2 * (deribit_value - iv) / (iv + deribit_value) * 100))
result = pd.DataFrame({"risk_free_rates": rs, "errors": errors})
# plotting
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=result["risk_free_rates"], y=result["errors"])
)
fig.update_layout(
    title=f"risk-free rate for {tick['symbol'][0:3]} option: {result[result['errors'] == result.errors.min()].iloc[0]['risk_free_rates']} "
)
fig.show()